# LangChain + Ollama: Resume RAG Agent Lab

This notebook demonstrates how to use Retrieval Augmented Generation (RAG) with a locally running Ollama model to answer interview questions based on your resume. You'll learn how to load a resume PDF, split it into chunks, and use Ollama to answer questions using the resume as context.

# Install Required Packages
To use LangChain with Ollama, you need to install the following Python packages:
- `ollama`: Python client for locally running Ollama models
- `langchain_community`: Community-contributed LangChain tools
- `python-dotenv`: For loading environment variables from a `.env` file
- `faiss-cpu`: For vector search
- `pypdf`: For PDF loading

In [ ]:
%pip install -r requirements.txt

# Import Libraries
We need to import the following libraries:
- `os`: For accessing environment variables
- `load_dotenv` from `dotenv`: To load variables from a `.env` file
- `ollama`: To interact with Ollama models
- `PyPDFLoader` from `langchain_community.document_loaders`: To load PDF
- `FAISS` from `langchain_community.vectorstores`: For vector search
- `RecursiveCharacterTextSplitter` from `langchain.text_splitter`: For chunking

In [ ]:
import os
from dotenv import load_dotenv
import ollama
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load Environment Variables
Environment variables are used to securely store sensitive information. We use `load_dotenv()` to load these variables from a `.env` file.

In [ ]:
load_dotenv()
ollama_model = os.getenv("OLLAMA_MODEL", "llama2")

# Load and Chunk Resume PDF
Load your resume PDF and split it into manageable chunks for semantic search.

In [ ]:
pdf_path = "my_resume.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(split_docs, embedding=None)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# Ask Interview Questions
Send a question to the model using the resume context. If the answer is not found, the model will say so.

In [ ]:
def ask_resume_question(question):
    docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in docs])
    prompt = f"Answer the following interview question using only the resume context below. If not found, say 'I don't know based on my resume.'\nResume:\n{context}\nQuestion: {question}"
    response = ollama.chat(model=ollama_model, messages=[{"role": "user", "content": prompt}])
    return response["message"]["content"]

question = "What are your key skills?"
print("Bot:", ask_resume_question(question))

---

*Try changing the question to see how the model answers using your resume!*